In [4]:
import numpy as np
import matplotlib.pyplot as plt

In [5]:
def f(x):
    return x*np.exp(x) + np.sin(np.exp(x))

def split(a, b, t):
    return a + t*(b-a)

def random_split(a, b):
    # Not exactly [0, 1] for better performance
    delta = 1e-1
    return split(a, b, np.random.uniform(delta, 1 - delta))

def optimize(f, a, b, t):
    x = split(a, b, t)
    fx = f(x)
    k = 1
    while b - a > eps:
        y = split(a, x, t)
        fy = f(y)
        k += 1
        if fy <= fx:
            b = x
            x = y
            fx = fy
        else:
            z = split(x, b, t)
            fz = f(z)
            k += 1
            if fz <= fx:
                a = x
                x = z
                fx = fz
            else:
                a = y
                b = z
    return k

def random_optimize(f, a, b, n):
    x = random_split(a, b)
    fx = f(x)
    k = 1
    while k <= n:
        y = random_split(a, x)
        fy = f(y)
        k += 1
        if fy <= fx:
            b = x
            x = y
            fx = fy
        else:
            z = random_split(x, b)
            fz = f(z)
            k += 1
            if fz <= fx:
                a = x
                x = z
                fx = fz
            else:
                a = y
                b = z
    return fx

# Минимум функции

Найдём минимум функции $$f(x) = x \cdot e^x + \sin e^x, \ x \in [-5, 0]$$ с помощью обычной дихотомии.

In [ ]:
a, b = -5, 0
t = 0.5
eps = 1e-7

def f(x):
    return x*np.exp(x) + np.sin(np.exp(x))

fig = plt.figure(figsize=(8.5, 6.5))

ax = fig.add_subplot(111)

xs = np.arange(a, b, 1e-2) 
ax.plot(xs, f(xs))

def split(a, b, t):
    return a + t*(b-a)

xs = np.arange(a, b, 1e-2) 
ax.plot(xs, f(xs))

fs = []
borders = []

import math
x = split(a, b, t)
fx = f(x)
# Amount of function calls
k = 1

borders.append([a, b])
while b - a > eps:
    fs.append(fx)
    borders.append([a, b])
    y = split(a, x, t)
    fy = f(y)
    k += 1
    if fy <= fx:
        b = x
        x = y
        fx = fy
    else:
        z = split(x, b, t)
        fz = f(z)
        k += 1
        if fz <= fx:
            a = x
            x = z
            fx = fz
        else:
            a = y
            b = z
print(f"Function calls: {k}")
print(f"Error achieved: {b-a}")
print(f"Minimum: f(x) = {fx:.2} at x = {x:.2}")

for i in range(len(borders)):
    ax.plot(borders[i], [0, 0], linewidth=10.0 * (i+1) / len(borders))    

ax.scatter(x, fx, s=50)

plt.tight_layout()
plt.grid()
plt.show()

# Зависимость от коэффициента пропорциональности

Найдём, при биении в каком отношении минимум находится наиболее быстро

In [ ]:
a, b = -5, 0
eps = 1e-7

fig = plt.figure(figsize=(8.5, 6.5))

ax = fig.add_subplot(111)
ax.set_title(r'Number of iterations depending on $t$ ($\varepsilon = 10^{-4}$)')
ax.set_xlabel(r't $(x_{new} = a + t\cdot(b-a))$')
ax.set_ylabel('Iterations')
ts = np.linspace(1e-1, 1 - 1e-1, 1000) 
ks = [optimize(f, a, b, t) for t in ts]
ax.plot(ts[::5], ks[::5])

z = np.polyfit(ts, ks, 2)
p = np.poly1d(z)
pts = p(ts)
plt.plot(ts, pts, "r--")

arg_min = np.argmin(ks)
apr_min = np.argmin(pts)
print(f"Minimum iterations done ({ks[arg_min]}) with t = {ts[arg_min]:.2}")
print(f"Iterations done with t = 0.5:\t{ks[len(ks)//2]}")
print(f"Approximated minimum iterations ({int(round(pts[apr_min]))}) \
with t = {ts[apr_min]:.2}")



plt.tight_layout()
plt.grid()
plt.show()

# Случайный выбор коэффициента пропорциональности

Всё-таки приводит к хорошим последствиям. В среднем сходится меньше, чем за 15 итераций, что не хуже, чем дихотомия (около 20 итераций для той же точности). Но, чтобы брать среднее, надо прогонять алгоритм несколько раз, плюс, добавляется время на генерацию случаных чисел. Так что данному алгоритму требуется около 30 итераций.

In [ ]:
a, b = -5, 0

fig = plt.figure(figsize=(8.5, 6.5))

ax = fig.add_subplot(111)
ax.set_title(r'Objective function value depending on iteration number')
ax.set_xlabel(r'Iterations')
ax.set_ylabel(r'Value')
its = np.linspace(0, 30, 30)
fs = np.array([[random_optimize(f, a, b, it) for it in its] for i in range(100)])

fs_mean = np.mean(fs, axis=0)
fs_std = np.std(fs, axis=0)
ax.plot(its, fs_mean)
ax.fill_between(its, fs_mean - fs_std, fs_mean + fs_std, alpha=0.2)

plt.tight_layout()
plt.grid()
plt.show()